#### TEACHER: Mike X Cohen, sincxpress.com
##### COURSE URL: udemy.com/course/deeplearning_x/?couponCode=202401

In [50]:
# import libraries
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

# Import and process the data

In [51]:
# import dataset (comes with colab!)
data = np.loadtxt(open('mnist_train_small .csv','rb'),delimiter=',')

# don't need labels!
data = data[:,1:]

# normalize the data to a range of [0 1]
dataNorm = data / np.max(data)

# convert to tensor
dataT = torch.tensor( dataNorm ).float()

# Create the DL model

In [52]:
# create a class for the model
def createTheMNISTAE( n_encode, n_bottleneck):

  class aenet(nn.Module):
    def __init__(self, n_encode, n_bottleneck):
      super().__init__()

      ### input layer
      self.input = nn.Linear(784,n_encode)
      
      ### encoder layer
      self.enc = nn.Linear(n_encode,n_bottleneck)

      ### latent layer
      self.lat = nn.Linear(n_bottleneck,n_encode)

      ### decoder layer
      self.dec = nn.Linear(n_encode,784)

    # forward pass
    def forward(self,x):
      x = F.relu( self.input(x) )
      x = F.relu( self.enc(x) )
      x = F.relu( self.lat(x) )
      y = torch.sigmoid( self.dec(x) )
      return y
  
  # create the model instance
  net = aenet(n_encode,n_bottleneck)
  
  # loss function
  lossfun = nn.MSELoss()

  # optimizer
  optimizer = torch.optim.Adam(net.parameters(),lr=.001)

  return net,lossfun,optimizer

# Create a function that trains the model

In [53]:
def function2trainTheModel(net, lossfun, optimizer):

  # number of epochs
  numepochs = 10000
  
  # initialize losses
  losses = torch.zeros(numepochs)

  # loop over epochs
  for epochi in range(numepochs):

    # select a random set of images
    randomidx = np.random.choice(dataT.shape[0],size=32)
    X = dataT[randomidx,:]

    # forward pass and loss
    yHat = net(X)
    loss = lossfun(yHat,X)

    # backprop
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # losses in this epoch
    losses[epochi] = loss.item()
  # end epochs

  # function output
  return losses,net

# Parametric experiments

In [54]:
# define the model parameters
n_encode = np.linspace(10, 500, 12, dtype=int)
n_bottleneck = np.linspace(5, 100, 8, dtype=int)

# Initialize a dictionary to store the results
results = {}

In [55]:
# start the experiment!
for encode_idx, current_n_encode in enumerate(n_encode):
    for bottleneck_idx, current_n_bottleneck in enumerate(n_bottleneck):

        # create and train a fresh model
        net, lossfun, optimizer = createTheMNISTAE(current_n_encode, current_n_bottleneck)
        losses, _ = function2trainTheModel(net, lossfun, optimizer)

        # store the average of the last 3 losses
        if len(losses) >= 3:
            avg_loss = torch.mean(losses[-3:]).item()
        else:
            avg_loss = torch.mean(losses).item()

        results[(current_n_encode, current_n_bottleneck)] = {'average_loss': avg_loss}

        # print a friendly status message
        print(f'Finished testing with n_encode={current_n_encode} and n_bottleneck={current_n_bottleneck}, Average Loss = {avg_loss:.4f}')

# You can now analyze the 'results' dictionary
print("\nAverage Loss for all combinations:")
for params, res in results.items():
    print(f"n_encode={params[0]}, n_bottleneck={params[1]}: Average Loss = {res['average_loss']:.4f}")


Finished testing with n_encode=10 and n_bottleneck=5, Average Loss = 0.0461
Finished testing with n_encode=10 and n_bottleneck=18, Average Loss = 0.0450
Finished testing with n_encode=10 and n_bottleneck=32, Average Loss = 0.0383
Finished testing with n_encode=10 and n_bottleneck=45, Average Loss = 0.0457
Finished testing with n_encode=10 and n_bottleneck=59, Average Loss = 0.0356
Finished testing with n_encode=10 and n_bottleneck=72, Average Loss = 0.0396
Finished testing with n_encode=10 and n_bottleneck=86, Average Loss = 0.0377
Finished testing with n_encode=10 and n_bottleneck=100, Average Loss = 0.0430
Finished testing with n_encode=54 and n_bottleneck=5, Average Loss = 0.0372
Finished testing with n_encode=54 and n_bottleneck=18, Average Loss = 0.0152
Finished testing with n_encode=54 and n_bottleneck=32, Average Loss = 0.0133
Finished testing with n_encode=54 and n_bottleneck=45, Average Loss = 0.0119
Finished testing with n_encode=54 and n_bottleneck=59, Average Loss = 0.0113
